# Face Detection And Recognition using OpenCV in Python 

In this Project we have complete whole program in such devided parts:
            1. Import all required modules 
            2. Create module to complete all necessoary tasks.(i.e Function)
            3. Creating DataSets using web cam
            4. Collecting Data (images)
            5. Recognition
               

## 1. Importing all modules

In [1]:
import cv2
import numpy as np

In [2]:
import os
import pickle
import matplotlib.pyplot as plt
from IPython.display import clear_output

## 2. Creating modules

###### To detect the face of person in web cam

In [3]:
def detect_face(frame):   
    detector = cv2.CascadeClassifier("frontal_face.xml")
    faces = detector.detectMultiScale(frame,1.2,5)
    return faces

###### Use to convert "color" images into "black & white" images

In [4]:
def gray_scale(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return image

###### use to cut main face and leave other part of person 

In [5]:
def cut_faces(image, faces_coord):
    faces = [] 
    for (x, y, w, h) in faces_coord:
        faces.append(image[y: y + h, x : x + w ]) #y is row..y+h number of rows,x is column x+w number of columns     
    return faces

###### Use to adjust brightness/intensity of image so that image will be more clean  
image is be taken may be in low light or high light place this wwill adjust and make a clean image

In [6]:
def normalize_intensity(images):
    images_norm = []
    for image in images:
        images_norm.append(cv2.equalizeHist(image))
    return images_norm

### Resize Image
 
 This module is use for zoom in Or zoom out the image so that cam feel comfort to take images

In [7]:
def resize(images,size=(47,62)):
    image_resize = []
    for image in images:
        if image.shape < size:
            img_size = cv2.resize(image,size,interpolation=cv2.INTER_CUBIC)
        else:
            img_size = cv2.resize(image,size,interpolation=cv2.INTER_AREA)
        image_resize.append(img_size)    
    return image_resize

### Make Pipeline of all Image Editing Functions:


To make image more clear and adjust intensity 

In [8]:
def normalize_faces(frame, faces_coord):
    faces = cut_faces(frame, faces_coord)
    faces = normalize_intensity(faces)
    faces = resize(faces)
    return faces

### Plotting Image 

This module is use to plot images in 2D (black & white) from 3D (colorfull)

In [9]:
def plot_show(image,title=""):
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.axis("off")
    plt.title(title)
    plt.imshow(image,cmap="Greys_r")
    plt.show()

### Border of Frame for Face Detection:

The border of rectanle marks on persons face in color with RGB combination (50,0,255)

In [10]:
def draw_rectangle(image, coords):
    for (x, y, w, h) in coords:
        cv2.rectangle(image, (x , y), (x + w , y + h), (50,0,255),2)

# 3. Create Datasets using web cam

This cell participate in our whole project to open cam and take images and count them and as we have give limitation of 100 img It count img and when it reach 100 it will auto-stop to taking image

In [11]:
count = int(input('Enter No. of Persons: '))

for i in range(1, count+1): #loop to count no of persons
    cam = cv2.VideoCapture(0)#web cam on
    folder = "faces/"+input('Person {}: '.format(i)).lower()
    
    if not os.path.exists(folder): #selecting path and folder
        os.mkdir(folder)
        flag_start_capturing = False
        sample=1
        cv2.namedWindow("Face", cv2.WINDOW_NORMAL)

        while True:
            ret,frame = cam.read()
            gray = gray_scale(frame)
            faces_coord = detect_face(gray)

            if len(faces_coord):
                faces = normalize_faces(gray,faces_coord)
                cv2.imwrite(folder + '/' + str(sample)+'.jpg',faces[0])
                plot_show(faces[0],"Capture Count:"+str(sample))
                clear_output(wait=True)
                if flag_start_capturing == True:
                    sample += 1
            
            draw_rectangle(frame,faces_coord)
            cv2.imshow('Face',frame)
            keypress=cv2.waitKey(1)
        
            if keypress == ord('c'):
                if flag_start_capturing == False:
                    flag_start_capturing = True
        
            if sample >100: # Capture Count of images
                break
            
            if keypress == ord('q'): # Quit ->>>> to close the cam press q
                break

        cam.release()
        cv2.destroyAllWindows()
    else:
        print ("This name already exists.")

Enter No. of Persons: 3
Person 1: adarshji
This name already exists.
Person 2: vinit1
This name already exists.
Person 3: anans
This name already exists.


In [12]:
cam.release()

## 4.  Collecting  Datasets

###### Using this cell we collect our all images in our created folder i.e. "faces"

In [13]:
def collect_dataset():
    images = []
    labels = []
    labels_dic = {}
   
    people = [Anans for Anans in os.listdir("faces/")]
   
    for i, Anans in enumerate(people):
        labels_dic[i] = Anans
        for image in os.listdir("faces/" + Anans):
            if image.endswith('.jpg'):
                images.append(cv2.imread("faces/" + Anans + '/' + image, 0))
                labels.append(i)
    return (images, np.array(labels), labels_dic)

In [14]:
images, labels, labels_dic = collect_dataset()

In [15]:
print (len(images))
print (labels_dic)

300
{0: '.ipynb_checkpoints', 1: 'adarshji', 2: 'anans', 3: 'vinit1'}


In [16]:
images[0].shape

(62, 47)

In [17]:
train = np.asarray(images)

In [18]:
print(type(train))
train.shape

<class 'numpy.ndarray'>


(300, 62, 47)

In [19]:
x_train = train.reshape(len(train),-1)

In [20]:
x_train.shape

(300, 2914)

In [21]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train_sc = sc.fit_transform(x_train.astype(np.float64))

In [22]:
from sklearn.decomposition import PCA
pca = PCA(n_components=.97)
x_train_pca = pca.fit_transform(x_train_sc)

In [23]:
pca.n_components_

101

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [25]:
param = {'C':[.001,.01,.1,1,10],'penalty':['l2','l1']}
gd_log = GridSearchCV(LogisticRegression(solver='liblinear'), param_grid = param, cv=5, scoring='accuracy')
gd_log.fit(x_train_pca,labels)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['l2', 'l1'], 'C': [0.001, 0.01, 0.1, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [26]:
print(gd_log.best_score_)
gd_log.best_params_

1.0


{'C': 0.001, 'penalty': 'l2'}

In [27]:
clf_log = gd_log.best_estimator_

In [28]:
filename = 'Face_Detection_log.pkl'
f = open(filename, 'wb')
pickle.dump(clf_log, f)
f.close()

In [29]:
filename = 'Face_Detection_log.pkl'
clf_log_load = pickle.load(open(filename, 'rb'))

In [30]:
clf_log_load

LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## 5. Face Recognition

###### recognise face of saved person like in this I use 3 persons named 1. Anans 2. Vinit1 3. Adarshji


In [31]:
cam = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_PLAIN
cv2.namedWindow("opencv_face", cv2.WINDOW_AUTOSIZE)

while True:
    ret,frame = cam.read()
    
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces_coord = detect_face(gray) # detect more than one face
    if len(faces_coord):
        faces = normalize_faces(gray, faces_coord)
        faces = normalize_intensity(faces)
        for i, face in enumerate(faces): # for each detected face
            x_test = face.reshape(1,-1)
            x_test_sc = sc.transform(x_test.astype(np.float64))
            x_test_pca = pca.transform(x_test_sc)    
            
            prob = gd_log.predict_proba(x_test_pca)
            confidence = gd_log.decision_function(x_test_pca)
            print (confidence)
            print (prob)
           
            y_pred = gd_log.predict(x_test_pca)
            print (y_pred, y_pred[0])
           
            name = labels_dic[y_pred[0]].capitalize()
            print(name)
            
            if prob[0][0]>.9:
                cv2.putText(frame, 'vivek', (faces_coord[i][0], faces_coord[i][1] - 10),
                            cv2.FONT_HERSHEY_PLAIN, 2, (66, 53, 243), 2)
                
            elif prob[0][1]>.85:
                cv2.putText(frame, 'anand', (faces_coord[i][0], faces_coord[i][1] - 10),
                            cv2.FONT_HERSHEY_PLAIN, 3, (66, 53, 243), 2)
                  
        clear_output(wait = True)
        draw_rectangle(frame, faces_coord) # rectangle around face
        
    cv2.putText(frame, "ESC to exit", (5, frame.shape[0] - 5),cv2.FONT_HERSHEY_PLAIN, 1.3, (66, 53, 243), 2,cv2.LINE_AA)
    
    cv2.imshow("opencv_face", frame) # live feed in external
    if cv2.waitKey(5) == 27:
        break
        
cam.release()
cv2.destroyAllWindows()
  

[[-0.44341354  4.10380667 -4.82970196]]
[[0.28274593 0.71152174 0.00573233]]
[2] 2
Anans


#            Thank you